In [5]:
import gurobipy as gp
print(gp.gurobi.version())

(12, 0, 1)


In [6]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

#load data
df_songs = pd.read_csv('songs_with_predictions_small.csv')
df_songs_sorted = df_songs.sort_values(by='average_prediction', ascending=False)

target_user = "user_5c0cba28e67a99c235ef2ba9877fef44"
ratings = df_songs[target_user].tolist()
track_names = df_songs['track_name'].tolist()
artists = df_songs['artist_name'].tolist()
unique_artists = list(set(artists))
num_tracks = len(track_names)

#create model
m = Model("Spotify_Base_Model")

#d.v. for each track & artist
x = m.addVars(num_tracks, vtype=GRB.BINARY, name="x")
a = m.addVars(unique_artists, vtype=GRB.BINARY, name="artist")

#constraint for at least 30 songs & link artist variables to selected songs
m.addConstr(quicksum(x[i] for i in range(num_tracks)) >= 30, "min_tracks")
for artist in unique_artists:
    song_indices = [i for i in range(num_tracks) if artists[i] == artist]
    m.addConstr(quicksum(x[i] for i in song_indices) <= 1, f"artist_{artist}")
    m.addConstr(a[artist] == quicksum(x[i] for i in song_indices), f"link_{artist}")

#objective to maximize total predicted rating
m.setObjective(quicksum(ratings[i] * x[i] for i in range(num_tracks)), GRB.MAXIMIZE)
m.optimize()

#extract selected tracks
selected_indices = [i for i in range(num_tracks) if x[i].X > 0.5]
selected_tracks = df_songs.iloc[selected_indices].copy().sort_values(by=target_user, ascending=False)
selected_tracks.head(30)

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14961 rows, 27480 columns and 67480 nonzeros
Model fingerprint: 0xc8365b73
Variable types: 0 continuous, 27480 integer (27480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-04, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective -2.0489554
Presolve removed 14961 rows and 27480 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.02 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 38.3913 -2.04896 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.839125143921e+01, best bound 3.839125143921e+01, gap 0.0000%


,track_name,artist_name,mbid,user_b4beed9bf653604a876fdfd9df59e19c,user_5c0cba28e67a99c235ef2ba9877fef44,user_3f8fd0d54f6f43ddce16c3d125dc640e,user_7c239807289a677c0ca4b220474dcf75,user_9c91d35a30b8fb61a04ef2711cb2c3b4,user_acb4965613fe484786942bba8b23ff24,user_8c13829fe6af09f5c7bcc492ef061c17,...,user_df486328a5745236ce5e9795b545ef33,user_52d6617fa785b088a0e058e9d7f394e5,user_9fba4899816193812d39288ccba4c603,user_9139c0d2effabdff39ef862e3c7289d1,user_a03045cd25d8a0a28a51c60fa7afe312,user_7391a216d4212b84c0a144479ddb1871,user_f9a8bd267b4030e0c2da5a99539bf1b6,user_5b96d1a747822774a342ea1b6d8a049b,user_ad3922719c2403f00f1bb9952e2666cf,average_prediction
19426,Lovers In Japan (Acoustic Version),Coldplay,cc197bad-dc9c-440d-a5b5-d52ba2e14234,-0.051434,1.329584,0.528964,0.932216,0.291404,-0.014872,0.094668,...,1.223740,1.890801,-0.036359,1.144039,1.743839,0.324575,0.362790,0.053370,-0.134987,0.658101
15794,Dj Skylat Dope Mix ft.Jamiroquai,Daft Punk,056e4f3e-d505-4dad-8ec1-d04f521cbb56,0.115311,1.245485,0.504059,0.994341,0.308111,0.209880,0.312735,...,1.411027,1.689002,0.080766,0.972412,1.667334,0.227663,0.526330,0.240038,0.002866,0.720671
19252,What's My Name (Kik Klap Extended Mix),Rihanna,73e5e69d-3554-40d8-8516-00cb38737a1c,-0.340553,1.238411,0.399119,0.603766,0.175094,-0.372603,-0.238584,...,1.005038,1.854097,-0.354208,1.056940,1.733482,0.206844,0.000136,-0.325568,-0.468260,0.416633
6699,The Glory - Album Version (Edited),Kanye West,164f0d73-1234-4e2c-8743-d77bf2191051,-0.030781,1.181904,0.579074,0.782051,0.255991,-0.089609,0.126528,...,1.120172,1.584544,-0.074627,1.102453,1.670100,0.209961,0.200556,-0.016486,-0.213617,0.578816
2343,It's your life (feat. Chris Willis),David Guetta,302bd7b9-d012-4360-897a-93b00c855680,-0.364708,1.138525,0.365423,0.513673,0.124154,-0.470158,-0.285294,...,0.991051,1.630709,-0.388150,1.049787,1.720530,0.159878,-0.094323,-0.399739,-0.497676,0.356594
16408,Work It Out - New Radio Edit,Beyoncé,859d0860-d480-4efd-970c-c05d5f1776b8,-0.440344,1.131092,0.357534,0.508884,0.116078,-0.432014,-0.307141,...,0.954467,1.742904,-0.374378,0.990812,1.588321,0.115392,-0.060255,-0.406074,-0.535313,0.337438
18590,3 a.m. (Produced By Dr. Dre),Eminem,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,-0.361912,1.101457,0.393204,0.520987,0.134177,-0.390637,-0.141329,...,0.932618,1.636315,-0.372750,1.010433,1.535193,0.143177,-0.074222,-0.310777,-0.499455,0.376960
3302,Out Of Goodbyes With Lady Antebellum,Maroon 5,0ab49580-c84f-44d4-875f-d83760ea2cfe,-0.574669,1.079023,0.190156,0.396669,0.028119,-0.639442,-0.463465,...,0.813192,1.533442,-0.537278,1.022848,1.646299,0.074680,-0.205547,-0.546291,-0.691130,0.220632
4648,Judas - DJ White Shadow Remix,Lady Gaga,650e7db6-b795-4eb5-a702-5ea2fc46c848,-0.570632,0.964962,0.166333,0.378225,-0.026052,-0.611735,-0.465825,...,0.899194,1.561162,-0.553460,0.911749,1.502690,0.005577,-0.213120,-0.515087,-0.691475,0.206122
16625,Dark Horse (feat. TEE),Katy Perry,122d63fc-8671-43e4-9752-34e846d62a9c,-0.549778,0.932020,0.193100,0.357009,-0.014682,-0.713830,-0.481862,...,0.764513,1.482571,-0.587266,0.822990,1.486859,0.004888,-0.266891,-0.603359,-0.691017,0.161712


In [7]:
print("Total tracks selected:", len(selected_indices))
print("Unique artists selected:", len(set(selected_tracks['artist_name'])))
print("Total predicted score:", selected_tracks[target_user].sum())

Total tracks selected: 4578
Unique artists selected: 4578
Total predicted score: 38.39125143921366
